In [33]:
import ipywidgets as widgets
from IPython.display import display
import collision_driver_models as cdm
import importlib
importlib.reload(cdm)

class ParametersWidget:
    def __init__(self, parameterizable, is_togglable = False, toggled_on = False):
        self.parameterizable = parameterizable
        self.is_togglable = is_togglable
        # header
        header_widget = widgets.HTML(value = '<b>' + parameterizable.name + '</b>')
        display(header_widget)
        # toggle
        if is_togglable:
            self.toggle = widgets.Checkbox(value = toggled_on, description = 'Active')
            self.toggle.style.description_width = '0'
            display(self.toggle)
        # parameters
        self.param_widgets = {}
        for i, param_def in enumerate(parameterizable.param_defs):
            param_val = parameterizable.param_vals[param_def.short_name]
            if param_def.type is cdm.ParameterType.BOOLEAN:
                param_widget = widgets.Checkbox(description = param_def.display_name, value = param_val)
                param_widget.style.description_width = '0'
                display(param_widget)
            elif param_def.type is cdm.ParameterType.COLOR:
                param_widget = widgets.ColorPicker(description = param_def.display_name, value = param_val)
                display(param_widget)
            else:
                min_val = param_def.range[0]
                max_val = param_def.range[1]
                if param_def.type is cdm.ParameterType.FLOAT:
                    param_widget = widgets.BoundedFloatText(min = min_val, max = max_val, value = param_val)
                else:
                    param_widget = widgets.BoundedIntText(min = min_val, max = max_val, value = param_val)
                hbox = widgets.HBox([widgets.Label(param_def.display_name + ' (' + param_def.unit + ')'), 
                                     param_widget])
                display(hbox)
            self.param_widgets[param_def.short_name] = param_widget
            
            

scenario = cdm.Scenario()
scenario.param_vals['v_E'] = 10
scenario.param_vals['T_g'] = 4
scenario.param_vals['T_o'] = 0
scenario.param_vals['W_P'] = 1.9
scenario.param_vals['v_P'] = 10
scenario.param_vals['T_a'] = 0
scenario.param_vals['a_P'] = -0.35

driver_models = []
fixed_ldt_model = cdm.FixedLDTModel()
fixed_ldt_model.param_vals['thetaDot_d'] = 0.002
driver_models.append(fixed_ldt_model)
mk2012_model = cdm.MaddoxAndKiefer2012Model()

sim_engine = cdm.SimulationEngine(scenario)
sim_engine.param_vals['end_time'] = 15
sim_engine.param_vals['n_simulations'] = 100
sim_engine.add_driver_model(fixed_ldt_model)


params_widgets = []
driver_model_widgets = []
params_widgets.append(ParametersWidget(scenario, False))
for driver_model in driver_models:
    driver_model_widget = ParametersWidget(driver_model, True)
    params_widgets.append(driver_model_widget)
    driver_model_widgets.append(driver_model_widget)
params_widgets.append(ParametersWidget(sim_engine, False))

output = widgets.Output()

def on_run_btn_clicked(b):
    # set parameter values
    for params_widget in params_widgets:
        for param_def in params_widget.parameterizable.param_defs:
            params_widget.parameterizable.param_vals[param_def.short_name] = (
                params_widget.param_widgets[param_def.short_name].value)
    # specify driver models to include
    sim_engine.clear_driver_models()
    for driver_model_widget in driver_model_widgets:
        if driver_model_widget.toggle.value:
            sim_engine.add_driver_model(driver_model_widget.parameterizable)
    # run simulations
    sim_engine.simulate_driver_models()
    # plot
    output.clear_output()
    with output:
        sim_engine.plot()

run_button = widgets.Button(description = 'Run')
run_button.on_click(on_run_btn_clicked)
display(run_button)
display(output)


HTML(value='<b>Scenario</b>')

HTML(value='<b>Looming detection threshold model</b>')

Checkbox(value=False, description='Active', style=DescriptionStyle(description_width='0'))

ColorPicker(value='magenta', description='Plot color')

HTML(value='<b>Simulation/plotting</b>')

Checkbox(value=True, description='Plot base scenario', style=DescriptionStyle(description_width='0'))

Button(description='Run', style=ButtonStyle())

Output()